This notebook reproduces the values of the standardisation parameters presented in Ginolin+24b. There might be slight variations of the values (within errorbars) due to the inclusion in Ginolin+24b of the very nearby SNe Ia ($z<0.015$) not included in the public ZTF DR2 release. The fitting procedure is here packaged into classes, but to have a better insight of how the `standax` code works, you can look at the notebooks `basic_standardisation_ztfsniadr2.ipynb`, `total_chi2.ipynb` and `totalchi2_and_standardisation.ipynb` at https://github.com/mginolin/standax/tree/main/notebooks.

In [1]:
import numpy as np
import pandas
from scipy.stats import norm
from astropy.stats import mad_std
import standax
from jax import numpy as jnp
import ztfcosmo

# Loading data

In [2]:
# Masterlist for SNe used in Ginolin+24a,b (available at https://github.com/mginolin/standax/)
g25ab_masterlist = pandas.read_csv('Ginolin25ab_masterlist.csv')

In [3]:
# Loading ZTF data
ztf_data = ztfcosmo.get_data(good_coverage=True, good_lcfit=True)
# Only keeping SNe in the master list
ztf_data = ztf_data[(ztf_data.index.isin(g25ab_masterlist['ztfname'])) & (ztf_data['redshift'] > 0.015)]

In [4]:
ztf_data

,redshift,redshift_err,source,t0,x0,x1,c,t0_err,x0_err,x1_err,...,dec_host,globalmass,globalmass_err,globalrestframe_gz,globalrestframe_gz_err,d_dlr,localmass,localmass_err,localrestframe_gz,localrestframe_gz_err
ztfname,,,,,,,,,,,,,,,,,,,,,
ZTF18aabyhlc,0.029055,0.000015,z_gal,58180.621262,0.001247,-2.747574,0.333068,0.504143,0.000122,0.091428,...,38.420393,10.003,0.120930,1.169697,0.019849,1.532268,7.405,0.263,0.732697,0.375373
ZTF18aagrtxs,0.029741,0.000011,z_gal,58213.150268,0.005583,-0.711100,-0.148609,0.041382,0.000170,0.037110,...,50.979166,10.583,0.154674,1.268697,0.018439,0.391638,9.062,0.057,1.272697,0.021260
ZTF18aagstdc,0.040367,0.003890,z_snid,58214.478445,0.004135,-0.047931,-0.097615,0.035815,0.000125,0.049832,...,42.088448,7.942,0.153154,1.082697,0.162111,0.146010,7.874,0.481,1.249697,1.128405
ZTF18aagtcxj,0.032359,0.000017,z_gal,58213.950331,0.001039,-1.883614,0.412887,0.043565,0.000034,0.046484,...,42.713852,10.268,0.120370,1.552697,0.018439,0.138965,9.870,0.062,1.678697,0.019105
ZTF18aahfbqp,0.041145,0.000013,z_gal,59208.836555,0.003087,-1.969449,-0.112340,0.204267,0.000171,0.278768,...,21.724007,10.879,0.136561,1.311697,0.018439,0.098646,10.543,0.118,1.295697,0.017692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF20adadffg,0.057145,0.004883,z_snid,59221.843973,0.001442,-0.568812,-0.079290,0.093281,0.000043,0.102160,...,42.762193,9.808,0.106977,1.027697,0.020616,1.184240,8.338,0.072,1.043697,0.101213
ZTF20adadsgm,0.043710,0.000015,z_gal,59218.758560,0.001794,-0.922459,0.007273,0.098082,0.000055,0.058838,...,56.465291,10.689,0.105622,1.311697,0.018439,0.728861,9.283,0.057,1.230697,0.020616
ZTF20adadshh,0.024565,0.000010,z_gal,59220.934923,0.003165,-0.772883,0.238502,0.175093,0.000111,0.093297,...,75.201326,10.340,0.101980,1.105697,0.019209,0.337090,9.936,0.095,1.331697,0.018439


In [5]:
# Adding a magnitude column (the zero point does not matter as we are only looking at residuals)
ztf_data['mag'] = -2.5*np.log10(ztf_data['x0']) + 25
ztf_data['mag_err'] = 2.5*ztf_data['x0_err']/(np.log(10)*ztf_data['x0'])

In [6]:
# SNe with redshift from host features
mask_hostz = ztf_data['source'].isin(['z_gal', 'z_nonSEDm'])

In [7]:
print(len(ztf_data[mask_hostz]))
print(len(ztf_data))

693
932


### Outlier rejection

In [8]:
a = standax.hubble.CleanDataset.from_dataset(ztf_data)
mask = a.reject_outliers() 
ztf_data_cut = ztf_data[mask]

In [9]:
print(len(ztf_data_cut))

925


# Standardisation

## Regular standardisation with the total-chi2 method from standax

### Full sample

The step is on local $(g-z)$ colour, and takes into account errors on colour. You can replace the parameter on which you perform the step by switching `localrestframe_gz` by `globalrestframe_gz`, `localmass` or `globalmass`.

In [10]:
bl = standax.hubble.HubbleResiduals.from_dataset(ztf_data_cut)
bl.set_block_hubble_const(False)
mask_blue = norm.cdf(1, loc=bl.data["localrestframe_gz"], scale=bl.data["localrestframe_gz_err"])
bl.set_stepcdf(mask_blue)
bl.fit_step(smooth_step=True, guess_beta=3.055, guess_alpha=0.161, guess_step=0.141)

In [11]:
print(bl.alpha, bl.alpha_err)
print(bl.beta, bl.beta_err)
print(bl.step, bl.step_err)

0.16083123 0.009549542040235826
3.0574682 0.05439480082926218
0.14116435 0.02189217770030163


### Only SNe with a redshift from host features

In [12]:
bl_hostz = standax.hubble.HubbleResiduals.from_dataset(ztf_data[mask & mask_hostz])
bl_hostz.set_block_hubble_const(False)
mask_blue = norm.cdf(1, loc=bl_hostz.data["localrestframe_gz"], scale=bl_hostz.data["localrestframe_gz_err"])
bl_hostz.set_stepcdf(mask_blue)
bl_hostz.fit_step(smooth_step=True)

In [13]:
print(bl_hostz.alpha, bl_hostz.alpha_err)
print(bl_hostz.beta, bl_hostz.beta_err)
print(bl_hostz.step, bl_hostz.step_err)

0.17319603 0.008942897729159677
3.0629475 0.050879889293846035
0.14016461 0.020248466267404522


Looking at the dispersion for the sample with all redshifts, and the sample with redshift coming from host features (~70% of the sample).

In [14]:
# nMAD dispersion (from astropy.stats.mad_std, less sensitive to outliers)
print(mad_std(bl.res_corr))
print(mad_std(bl_hostz.res_corr))

0.17568359
0.16345394


In [15]:
# Regular STD
print(np.std(bl.res_corr))
print(np.std(bl_hostz.res_corr))

0.22424571
0.20951256


## Broken-$\alpha$ standardisation

In [16]:
bl_broken = standax.hubble.HubbleResiduals.from_dataset(ztf_data_cut)
bl_broken.set_block_hubble_const(False)
mask_blue = norm.cdf(1, loc=bl_broken.data["localrestframe_gz"], scale=bl_broken.data["localrestframe_gz_err"])
bl_broken.set_stepcdf(mask_blue)
bl_broken.fit_broken_alpha(guess_sigma=0.15, smooth_step=True, force_sigmaint=False)

In [17]:
print(bl_broken.alpha, bl_broken.alpha_err)
print(bl_broken.x1break, bl_broken.x1break_err)
print(bl_broken.beta, bl_broken.beta_err)
print(bl_broken.step, bl_broken.step_err)

[Array(0.26895937, dtype=float32), Array(0.07897715, dtype=float32)] [np.float64(0.011369049720191027), np.float64(0.010274620647527207)]
-0.47794396 0.07790712
3.2683122 0.028836189606424135
0.16492689 0.01259211228329771


## A posteriori fit for the step
This method corresponds to the pink points in Fig.10 of Ginolin+24b

In [18]:
# First fit for alpha and beta
bl_post = standax.hubble.HubbleResiduals.from_dataset(ztf_data_cut)
bl_post.set_block_hubble_const(False)

In [19]:
print(bl_post.alpha, bl_post.alpha_err)
print(bl_post.beta, bl_post.beta_err)
print(bl_post.step, bl_post.step_err)

0.13035455 0.008334006578629522
2.9923112 0.06138569497223602
None None


In [20]:
# Then compute the step as an posteriori correction

stepcdf = norm.cdf(1, loc=bl.data["localrestframe_gz"], scale=bl.data["localrestframe_gz_err"])
step = np.average(bl_post.res_corr, weights=stepcdf)-np.average(bl_post.res_corr, weights=1-stepcdf)
err1 = np.sqrt(np.average((bl_post.res_corr-np.average(bl_post.res_corr, weights=stepcdf))**2, weights=stepcdf))/np.sqrt(len(bl_post.res_corr))
err2 = np.sqrt(np.average((bl_post.res_corr-np.average(bl_post.res_corr, weights=1-stepcdf))**2, weights=1-stepcdf))/np.sqrt(len(bl_post.res_corr))
step_err = np.sqrt(err1**2+err2**2)

In [21]:
print(step, step_err)

0.08736803296530599 0.010446394404445641
